In [21]:
# ============================================
# ProgettoDL_FootballSegmentation — Video → Frames
# Notebook dedicato ESCLUSIVAMENTE alla frammentazione video
# Pipeline: (1) Setup → (2) Percorsi → (3) Selezione video → (4) Estrazione frame
# ============================================

# ---------- (1) Setup e import librerie ----------
from pathlib import Path
import sys
import cv2
import re
from tkinter import Tk, filedialog
from utils.postprocessing import save_image   # serve solo per salvare i frame

try:
    from tqdm import tqdm
except ImportError:
    tqdm = lambda x, **k: x  # no-op se tqdm non disponibile

from IPython.display import display, Markdown

In [22]:
# ---------- (2) Definizione percorsi ----------
BASE_DIR = Path().resolve()                   # cartella dove si trova il notebook
DATA_INPUTS_DIR = BASE_DIR / "data_inputs"    # data_inputs allo stesso livello del notebook
DATA_INPUTS_DIR.mkdir(parents=True, exist_ok=True)

# Consenti l'import di utils/
sys.path.append(str(BASE_DIR.parent))

In [23]:
# ---------- (3) Selezione video da frammentare con finestra di dialogo ----------
Tk().withdraw()  # nasconde la finestra principale Tkinter
video_path_str = filedialog.askopenfilename(
    title="Seleziona il video",
    filetypes=[("Video files", "*.mp4 *.mov *.mkv *.avi"), ("All files", "*.*")]
)
if not video_path_str:
    raise RuntimeError("Nessun file selezionato.")

video_path = Path(video_path_str)

# ricava clip_id dal nome del file (es. clip01.mp4 -> '01')
m = re.search(r"clip(\d+)", video_path.stem, re.IGNORECASE)
clip_id = m.group(1) if m else "00"

# cartella dei frame estratti
frames_dir = DATA_INPUTS_DIR / f"extracted_frames_clip{clip_id}"
frames_dir.mkdir(parents=True, exist_ok=True)

In [24]:
# ---------- (4) Estrazione frame (barra + lista dinamica coerenti) ----------
cap = cv2.VideoCapture(str(video_path))
if not cap.isOpened():
    raise RuntimeError(f"Impossibile aprire il video: {video_path}")

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) if cap.get(cv2.CAP_PROP_FRAME_COUNT) > 0 else None

written_names = []  # per la lista dinamica dei frame estratti
extract_handle = display(Markdown("_Inizio estrazione frame…_"), display_id=True)

idx = 1
for _ in tqdm(range(frame_count) if frame_count else iter(int, 1), desc="Estrazione frame", unit="frame", total=frame_count if frame_count else None):
    ret, frame_bgr = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    name = f"frame{idx:02d}_clip{clip_id}.png"
    out_path = frames_dir / name
    save_image(frame_rgb, str(out_path))
    written_names.append(name)

    # Aggiorna lista dinamica (ultimi 10)
    last_w = written_names[-10:]
    extract_handle.update(Markdown(
        "**Frame estratti (ultimi 10):**  \n" + "<br>".join(last_w) + f"<br><br>Totale: **{len(written_names)}**"
    ))

    idx += 1

cap.release()
tot = idx - 1
if tot == 0:
    raise RuntimeError("Nessun frame estratto dal video.")

**Frame estratti (ultimi 10):**  
frame97_clip06.png<br>frame98_clip06.png<br>frame99_clip06.png<br>frame100_clip06.png<br>frame101_clip06.png<br>frame102_clip06.png<br>frame103_clip06.png<br>frame104_clip06.png<br>frame105_clip06.png<br>frame106_clip06.png<br><br>Totale: **106**

Estrazione frame: 100%|██████████| 106/106 [00:53<00:00,  1.97frame/s]


In [25]:
# ---------- (5) Report ----------
print(f"✅ Frammentazione completata. \n- Video: {video_path}\n- Frame salvati in: {frames_dir}\n- Totale frames estratti da clip{clip_id}: {idx-1}")

# ============================================
# Fine notebook frammentazione
# ============================================

✅ Frammentazione completata. 
- Video: C:\Users\ricks\Desktop\ProgettoDL\Football_match_segmentation\data_inputs\clip06.mp4
- Frame salvati in: C:\Users\ricks\Desktop\ProgettoDL\Football_match_segmentation\data_inputs\extracted_frames_clip06
- Totale frames estratti da clip06: 106
